In [46]:
# Agregamos el path para poder importar módulos personalizados
import sys
sys.path.append('/home/jovyan/work')

In [47]:
from src.data_loader import cargar_dataset
from collections import Counter
import torch
from torch.utils.data import DataLoader

#### **CARGAR DATOS**

In [8]:
# Cargamos el split de entrenamiento del dataset WikiText-103
dataset = cargar_dataset(split="train")

# Visualizamos la estructura básica
print("Estructura del dataset:\n", dataset)

Estructura del dataset:
 Dataset({
    features: ['text'],
    num_rows: 1801350
})


#### **VISUALIZACIÓN DE CONTENIDO DEL DATASET**

In [9]:
# Mostramos las primeras 5 líneas no vacías del dataset
count = 0
sentences = []
for sentence in dataset:
    if sentence["text"].strip():
        sentences.append(sentence["text"].strip())
        print("\nTexto:\n", sentence)
        count += 1
    if count == 5:
        break


Texto:
 {'text': ' = Valkyria Chronicles III = \n'}

Texto:
 {'text': ' Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . \n'}

Texto:
 {'text': " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments

#### **TOKENIZACIÓN Y VOCABULARIO MANUAL**

In [13]:
# Definimos un tokenizador básico que separa por espacios y pasa todo a minúsculas
def custom_tokenizer(text):
    return text.lower().split()

# Aplicamos tokenización a las 5 oraciones capturadas
for i, s in enumerate(sentences):
    tokens = custom_tokenizer(s)
    print(f"\nTexto {i+1}: {tokens}")

# Construimos un vocabulario manual usando Counter
word_vocab = Counter()

for s in sentences:
    tokens = custom_tokenizer(s)
    word_vocab.update(tokens)

# Mostramos el tamaño del vocabulario y las palabras más frecuentes
print(f"\nTotal de palabras en el vocabulario: {len(word_vocab)}")
print("\nVocabulario inicial (manual):")
for word, freq in word_vocab.most_common():
    print(f"'{word}': {freq}")


Texto 1: ['=', 'valkyria', 'chronicles', 'iii', '=']

Texto 2: ['senjō', 'no', 'valkyria', '3', ':', '<unk>', 'chronicles', '(', 'japanese', ':', '戦場のヴァルキュリア3', ',', 'lit', '.', 'valkyria', 'of', 'the', 'battlefield', '3', ')', ',', 'commonly', 'referred', 'to', 'as', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', ',', 'is', 'a', 'tactical', 'role', '@-@', 'playing', 'video', 'game', 'developed', 'by', 'sega', 'and', 'media.vision', 'for', 'the', 'playstation', 'portable', '.', 'released', 'in', 'january', '2011', 'in', 'japan', ',', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'valkyria', 'series', '.', 'employing', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', '@-@', 'time', 'gameplay', 'as', 'its', 'predecessors', ',', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the', '"', 'nameless', '"', ',', 'a', 'penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 'gallia', 'during', 'the', 'second', 'europan', 'war', 'who

#### **TOKENIZACIÓN Y VOCABULARIO USANDO TORCHTEXT**

##### **Definición de tokenizador y construcción de vocabulario**

In [20]:
from torch.utils.data import Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Tokenizador
tokenizer = get_tokenizer("basic_english")

# Construir el vocabulario
vocab = build_vocab_from_iterator(map(tokenizer, sentences))

print(f"Vocabulario construido con {len(vocab)} tokens")

Vocabulario construido con 171 tokens


##### **Definición de conjunto de datos personalizado**

In [22]:
# Definir un conjunto de datos personalizado
class CustomDataset(Dataset):
    def __init__(self, sentences, tokenizer, vocab):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.vocab = vocab

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokens = self.tokenizer(self.sentences[idx])
        # Convertir tokens a índices de tensor usando el vocabulario
        tensor_indices = [self.vocab[token] for token in tokens]
        return torch.tensor(tensor_indices)
    
# Crear una instancia de tu conjunto de datos personalizado
custom_dataset = CustomDataset(sentences, tokenizer, vocab)

print("Longitud del conjunto de datos personalizado:", len(custom_dataset))
print("Items de muestra:")

for i in range(5):
    sample_item = custom_dataset[i]
    print(f"Item {i + 1}: {sample_item}")


Longitud del conjunto de datos personalizado: 5
Items de muestra:
Item 1: tensor([ 9,  3,  6, 22,  9])
Item 2: tensor([148, 110,   3,  24,  17,   6,  46,  35, 170,   2,  96,   1,   3,   4,
          0,  59,  24,  47,   2,  65, 134,  15,  18,   3,   6,  22, 117,  23,
          2,  34,  11,  42, 140,  25, 124,  44,   7,  71,  19, 147,   5,  37,
          1,  45,  20,   0,  38, 125,   1,  39,   8,  95,  49,   8,  23,   2,
         10,  34,   0, 158,   7,   8,   0,   3,  14,   1,  77,   0, 143,  87,
          4,  42,   5, 132,  25, 159,  33,  18,  94, 129,   2,   0, 151, 141,
        120,  15,   0,  82,   7,   5,  83,   0, 107,   2,  11, 121, 103,  43,
        149,   0, 108,   4,  88,  76,   0, 145,  79, 162, 165, 122, 146,  61,
        115,   5,  57, 123,  55,   0,  92,  43,  17, 131,   1])
Item 3: tensor([  0,   7,  60,  30,   8,  48,   2,  63, 118,  11,  36, 126,   4,   0,
        166,  73, 113,   3,   6,  21,   1, 164,  10, 136,   0, 150,  81,   4,
          0,  14,   2,  10,  27, 161,

##### **Reconstrucción de texto a partir de índices**

In [25]:
# Función para convertir índices de vuelta a palabras
def indices_a_palabras(indices, vocab):
    return [vocab.get_itos()[idx] for idx in indices]

# Mostramos los textos reconstruidos
print("\nTextos procesados (reconstruidos):")

for i in range(5):
    sample_item = custom_dataset[i]
    palabras = indices_a_palabras(sample_item.tolist(), vocab)
    print(f"\nTexto {i+1}: {palabras}")


Textos procesados (reconstruidos):

Texto 1: ['=', 'valkyria', 'chronicles', 'iii', '=']

Texto 2: ['senjō', 'no', 'valkyria', '3', '<unk>', 'chronicles', '(', 'japanese', '戦場のヴァルキュリア3', ',', 'lit', '.', 'valkyria', 'of', 'the', 'battlefield', '3', ')', ',', 'commonly', 'referred', 'to', 'as', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', ',', 'is', 'a', 'tactical', 'role', '@-@', 'playing', 'video', 'game', 'developed', 'by', 'sega', 'and', 'media', '.', 'vision', 'for', 'the', 'playstation', 'portable', '.', 'released', 'in', 'january', '2011', 'in', 'japan', ',', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'valkyria', 'series', '.', 'employing', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', '@-@', 'time', 'gameplay', 'as', 'its', 'predecessors', ',', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the', 'nameless', ',', 'a', 'penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 'gallia', 'during', 'the', 'second'

##### **Cálculo de palabras más frecuentes**

In [27]:
# Calculamos la frecuencia de las palabras usando el vocabulario de torchtext
palabras_frecuentes = {}

for sentence in sentences:
    tokens = tokenizer(sentence)
    for token in tokens:
        if token in vocab.get_stoi():
            palabras_frecuentes[token] = palabras_frecuentes.get(token, 0) + 1

# Mostramos el vocabulario y su frecuencia
print(f"Total de palabras en el vocabulario: {len(palabras_frecuentes)}")
print("Palabras más frecuentes:")

for palabra, frecuencia in sorted(palabras_frecuentes.items(), key=lambda x: x[1], reverse=True):
    print(f"'{palabra}': {frecuencia}")

Total de palabras en el vocabulario: 171
Palabras más frecuentes:
'the': 22
'.': 16
',': 15
'valkyria': 10
'of': 9
'and': 8
'chronicles': 7
'game': 7
'in': 7
'=': 6
'it': 6
'a': 5
'with': 5
'was': 5
'to': 4
'series': 4
'iii': 3
'<unk>': 3
'as': 3
'japan': 3
'by': 3
'for': 3
'ii': 3
''': 3
'3': 2
'japanese': 2
'is': 2
'tactical': 2
'@-@': 2
'video': 2
'media': 2
'vision': 2
'playstation': 2
'released': 2
'gameplay': 2
'unit': 2
'development': 2
'large': 2
'also': 2
'both': 2
'along': 2
's': 2
'sales': 2
'an': 2
'expanded': 2
'edition': 2
'senjō': 1
'no': 1
'(': 1
'戦場のヴァルキュリア3': 1
'lit': 1
'battlefield': 1
')': 1
'commonly': 1
'referred': 1
'outside': 1
'role': 1
'playing': 1
'developed': 1
'sega': 1
'portable': 1
'january': 1
'2011': 1
'third': 1
'employing': 1
'same': 1
'fusion': 1
'real': 1
'time': 1
'its': 1
'predecessors': 1
'story': 1
'runs': 1
'parallel': 1
'first': 1
'follows': 1
'nameless': 1
'penal': 1
'military': 1
'serving': 1
'nation': 1
'gallia': 1
'during': 1
'second': 1
'

#### **TOKENIZACIÓN Y VOCABULARIO MANUAL A NIVEL DE CARACTERES**

In [44]:
# Creamos una lista donde cada oración está representada como una lista de caracteres en minúscula
tokenized_sentences_by_char = [list(sentence.lower().strip()) for sentence in sentences]

# Mostramos ejemplo de una oración tokenizada
for i in range(5):
    print(f"\nTexto {i+1}: {sentences[i]}")
    print(f"Tokenizada por caracteres {i+1}: {tokenized_sentences_by_char[i]}")


Texto 1: = Valkyria Chronicles III =
Tokenizada por caracteres 1: ['=', ' ', 'v', 'a', 'l', 'k', 'y', 'r', 'i', 'a', ' ', 'c', 'h', 'r', 'o', 'n', 'i', 'c', 'l', 'e', 's', ' ', 'i', 'i', 'i', ' ', '=']

Texto 2: Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " .
Tokenizada por caracteres 2: ['s', 'e', 'n', 'j', 'ō', ' ', 'n', 'o', ' ', 'v', 'a', 'l', '

In [45]:
# Construimos un vocabulario a partir de los caracteres tokenizados
char_vocab = Counter()

for sentence in tokenized_sentences_by_char:
    char_vocab.update(sentence)

# Mostramos el tamaño del vocabulario y los caracteres más frecuentes
print(f"\nTotal de caracteres únicos en el vocabulario: {len(char_vocab)}")
print("\nVocabulario inicial (caracteres):")
for char, freq in char_vocab.most_common():
    print(f"'{char}': {freq}")


Total de caracteres únicos en el vocabulario: 54

Vocabulario inicial (caracteres):
' ': 324
'e': 168
'a': 149
'i': 130
't': 108
'n': 104
'o': 96
'r': 95
's': 84
'l': 72
'd': 52
'h': 49
'c': 38
'm': 38
'p': 35
'g': 30
'v': 26
'y': 26
'u': 26
'f': 24
'w': 23
'k': 18
'.': 16
',': 15
'b': 13
'j': 9
'=': 6
'@': 4
'0': 4
'1': 4
'"': 4
'3': 3
':': 3
'<': 3
'>': 3
'2': 3
'z': 3
''': 3
'-': 2
'x': 2
'4': 2
'ō': 1
'(': 1
'戦': 1
'場': 1
'の': 1
'ヴ': 1
'ァ': 1
'ル': 1
'キ': 1
'ュ': 1
'リ': 1
'ア': 1
')': 1
